In [27]:
from typing import TypeAlias

mas: TypeAlias = float
"""Milliarcseconds"""
ly: TypeAlias = float
"""Light years"""

_ = _

In [36]:
def parallax_to_distance(parallax: mas) -> ly:
    return (1 / (parallax / 1000)) * 3.262


class Destination:
    def __init__(self, name: str, messier_number: int | None, object_type: str, parallax: mas, parallax_error: mas | None = None):
        self.name = name
        self.messier_number = messier_number
        self.object_type = object_type
        self.parallax: mas = parallax
        self.parallax_error: mas | None = parallax_error

        self.distance_ly = parallax_to_distance(parallax)
        if parallax_error is not None:
            dist_a = parallax_to_distance(parallax + parallax_error)
            dist_b = parallax_to_distance(parallax - parallax_error)
            self.distance_ly_error = abs((dist_a - dist_b) / 2)
        else:
            self.distance_ly_error = None
    
    def __str__(self):
        msr = f" (M{self.messier_number})" if self.messier_number else ""
        ly_err = f" ± {self.distance_ly_error:.3f}" if self.distance_ly_error else ""
        return f"{self.name}{msr} ({self.object_type}) - {self.distance_ly:.2f}{ly_err} ly"


destinations = [
    Destination("Crab Nebula", 1, "Planetary Nebula", 0.51104, 0.07876),
    Destination("Lagoon Nebula", 8, "Emission Nebula", 0.8505, 0.0952),
    Destination("Ring Nebula", 57, "Planetary Nebula", 1.2696, 0.0438),
    Destination("Alpha Centauri", None, "Star System", 750.81, 0.38),
    Destination("PSR B1257+12", None, "Pulsar", 1.41, 0.08),
]

print("Destinations:")
for dest in destinations:
    print(f"\t{dest}")

Destinations:
	Crab Nebula (M1) (Planetary Nebula) - 6383.06 ± 1007.673 ly
	Lagoon Nebula (M8) (Emission Nebula) - 3835.39 ± 434.758 ly
	Ring Nebula (M57) (Planetary Nebula) - 2569.31 ± 88.744 ly
	Alpha Centauri (Star System) - 4.34 ± 0.002 ly
	PSR B1257+12 (Pulsar) - 2313.48 ± 131.685 ly
